**1) Connect google drive**

In [ ]:
# Check if NVIDIA GPU is enabled
!nvidia-smi

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

**2) Clone the Darknet repo**



In [ ]:
!git clone https://github.com/AlexeyAB/darknet

**3) Compile Darknet for Nvidia GPU**


In [ ]:
# change makefile to have GPU and OPENCV enabled
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!make

**4) Configure Darknet for YOLO V3**

In [5]:
!cp cfg/yolov3.cfg cfg/yolov3_training.cfg

In [6]:
!sed -i 's/batch=1/batch=64/' cfg/yolov3_training.cfg
!sed -i 's/subdivisions=1/subdivisions=64/' cfg/yolov3_training.cfg
# !sed -i 's/max_batches = 500200/max_batches = 4000/' cfg/yolov3_training.cfg
!sed -i '610 s@classes=80@classes=10@' cfg/yolov3_training.cfg 
!sed -i '696 s@classes=80@classes=10@' cfg/yolov3_training.cfg
!sed -i '783 s@classes=80@classes=10@' cfg/yolov3_training.cfg 
!sed -i '603 s@filters=255@filters=45@' cfg/yolov3_training.cfg # filter = (classes + 5) * 3
!sed -i '689 s@filters=255@filters=45@' cfg/yolov3_training.cfg
!sed -i '776 s@filters=255@filters=45@' cfg/yolov3_training.cfg

In [ ]:
# Create folder on google drive to save the weights
!mkdir "/mydrive/cloudspotting"

In [8]:
!echo "altocumulus\naltostratus\ncirrocumulus\ncirrostratus\ncirrus\ncumulonimbus\ncumulus\nnimbostratus\nstratocumulus\nstratus" > data/obj.names # EDIT!
!echo -e 'classes=10\ntrain  = data/train.txt\nvalid  = data/test.txt\nnames = data/obj.names\nbackup = /mydrive/cloudspotting' > data/obj.data # EDIT
!mkdir data/obj

In [ ]:
# Download weights darknet model 53
!wget https://pjreddie.com/media/files/darknet53.conv.74

**5) Extract Images**

In [ ]:
!unzip /mydrive/images.zip -d data/obj

In [ ]:
import glob
images_list = glob.glob("data/obj/images/*.jpg")
print(images_list)

In [12]:
#Create training.txt file
file = open("data/train.txt", "w") 
file.write("\n".join(images_list)) 
file.close() 

**6) Train model**

In [ ]:
!./darknet detector train data/obj.data cfg/yolov3_training.cfg darknet53.conv.74 -dont_show